In [1]:
# Import the relevant python libraries for the analysis
import pandas as pd
from pandas import DataFrame
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import scipy.stats as stats
import pylab as pl
import math

In [2]:
# Load the dataset
materna = pd.read_csv('mortalidad_materna.csv')
#materna.info()

## Question: Are the differences in Average Age and Distribution of Age of Maternal Death Across Mexico's States/Providences Statistically Significant?

A One-Way Analysis of Variance (ANOVA) test compares the means of two or more groups to determine if at least one group mean is statistically different from the others. These assumptions that must be satisfied in order for the associated p-value to be valid:

1. The samples are independent.
2. Each sample is from a normally distributed population.
3. The population standard deviations of the groups are all equal. (homoscedasticity)

#### Step 2: Assess if the distribution of age of maternal death within each region is Normally Distributed.
- Create Empirical and Theoretical CDFs

In [3]:
# Create an Empirical Cumulative Distribution Function (ECDF)
def ecdf(data):
    """Compute ECDF for a one-dimensional array of measurements."""
    # x-data for the ECDF: x
    x = np.sort(data)

    # y-data for the ECDF: y
    y = np.arange(1, len(data)+1) / len(data)

    return x, y

In [4]:
# Compute the ECDF of data
x_death, y_death = ecdf(age_death)

# Seed the random number generator:
np.random.seed(15)

# Compute the theoretical CDF 
cdf_mean = np.mean(age_death)
cdf_std = np.std(age_death)
cdf_samples = np.random.normal(cdf_mean, cdf_std, size=10000)
x_norm, y_norm = ecdf(cdf_samples)

NameError: name 'age_death' is not defined

## One-Way Analysis of Variance

$H$o: All mean age of death of maternal women within all Province of Mexico are statistically similar. <br>  Empirical Mean ($μ$) − Population Mean ($μ$) = 0

$H$a: There is at least one mean age of death of maternal women within a Province of Mexico that is statistically different from the rest. <br> Empirical Mean ($μ$) − Population Mean ($μ$) ≠ 0

Significance Level: *95%* Confidence. <br> $α$ = 0.05